In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import display, HTML
from IPython.lib.pretty import pprint
from itertools import chain, tee

In [3]:
from mockdown.model import View
from mockdown.display.view import display_view

In [4]:
sample_view = View("p", (0, 0, 100, 100), children=[
    View("a1", (10, 10, 45, 90), children=[
        View("b11", (20, 20, 35, 45)),
        View("b12", (20, 55, 35, 80))
    ]),
    View("a2", (55, 10, 90, 90), children=[
        View("b21", (65, 20, 80, 45)),
        View("b22", (65, 55, 80, 80))
    ])
])

In [5]:
html_str = str(display_view(sample_view, scale=3))
display(HTML(html_str))

In [6]:
from mockdown.visibility import visible_pairs

edge_pairs = visible_pairs(sample_view, deep=True)
anchor_pairs = [(e1.anchor, e2.anchor) for (e1, e2) in edge_pairs]

In [7]:
html_str = str(display_view(sample_view, visible_pairs=edge_pairs, scale=3))
display(HTML(html_str))

In [8]:
from mockdown.logic import valid_constraints
constraints = list(valid_constraints(sample_view, anchor_pairs))

ImportError: cannot import name 'constrainable_pairs' from 'mockdown.logic' (/Users/dylan/Projects/mockdown/mockdown/logic.py)

In [ ]:
html_str = str(display_view(sample_view, constrainable_pairs=constrainable_pairs, scale=5))
display(HTML(html_str))

In [ ]:
from mockdown.daikon import generate_decls, generate_dtrace

decls_text = generate_decls("sample_view", constrainable_pairs)
dtrace_text = generate_dtrace("sample_view", constrainable_pairs)

print(decls_text[0:150] + "\n...\n")
print(dtrace_text[0:80] + "\n...\n")

In [ ]:
DAIKONJAR="/usr/local/opt/daikon/daikon-src/daikon.jar"
INVARIANTJAR="/Users/dylan/Projects/Pipeline/MockdownInvariants/out/artifacts/mockdown_inv/mockdown-inv.jar"
LIBJARS="/Users/dylan/Projects/Pipeline/MockdownInvariants/lib/*"

from asyncio import subprocess
from tempfile import NamedTemporaryFile
from shlex import quote
import platform

def mk_tmp(suffix: str):
    return NamedTemporaryFile(mode='w+', 
                              prefix="mockdown-",
                              suffix=suffix)

with mk_tmp('.decls') as decls_file, mk_tmp('.dtrace') as dtrace_file:
    decls_file.write(decls_text)
    decls_file.flush()
    
    dtrace_file.write(dtrace_text)
    dtrace_file.flush()

    cmd = (
        f"java -cp {DAIKONJAR}:{INVARIANTJAR}:{LIBJARS} daikon.Daikon"
        " --nohierarchy"
        " --disable-all-invariants"
        " --user-defined-invariant edu.ucsd.mockdown.inv.PositionSpacingUpper"
        " --config_option edu.ucsd.mockdown.inv.PositionSpacingUpper.enabled=true"
        f" {quote(decls_file.name)}"
        f" {quote(dtrace_file.name)}"
        " -o /dev/null"  # don't write a basename.inv.gz
    )
    
    print(f"> {cmd}\n")
    
    proc = await subprocess.create_subprocess_shell(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    await proc.wait()
    stdout, stderr = await proc.communicate()

    print(f"\x1b[32m{stdout.decode()}\x1b[0m")
    print(f"\x1b[31m{stderr.decode()}\x1b[0m")

In [ ]:
from mockdown.constraint.base import AlignmentConstraint